In [162]:
import requests
import urllib3
import secrets
import pandas as pd
import polyline

In [163]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Strava API Request

In [164]:
auth_url = 'https://www.strava.com/oauth/token'
activities_url = 'https://www.strava.com/api/v3/athlete/activities'

print('Requesting Strava token... \n')
res = requests.post(auth_url, data=secrets.strava_payload, verify=False)
strava_access_token = res.json()['access_token']

header = {'Authorization': 'Bearer ' + strava_access_token}

strava_requests_page_num = 1
all_activities = []

while True:
    strava_param = {'per_page' : 15, 'page' : strava_requests_page_num}
    strava_dataset = requests.get(activities_url, headers=header, params=strava_param).json()

    if len(strava_dataset) == 0:
        print('breaking out of Strava while loop because the response is zero, indicating no more activities.')
        break

    if all_activities:
        print('all activities is populated')
        all_activities.extend(strava_dataset)

    else:
        print('all activities is NOT populated')
        all_activities = strava_dataset

    strava_requests_page_num += 1

print('Total Activities: ', len(all_activities))

Requesting Strava token... 



all activities is NOT populated
all activities is populated
all activities is populated
all activities is populated
breaking out of Strava while loop because the response is zero, indicating no more activities.
Total Activities:  53


## Creating pandas dataframe for running data

In [165]:
run_data = pd.DataFrame(data=all_activities)

# Many Columns are not filled with information because there is no use of a watch or heart monitor.
- data can be cleaned

In [166]:
run_data.columns

Index(['resource_state', 'athlete', 'name', 'distance', 'moving_time',
       'elapsed_time', 'total_elevation_gain', 'type', 'sport_type',
       'workout_type', 'id', 'start_date', 'start_date_local', 'timezone',
       'utc_offset', 'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'map', 'trainer', 'commute', 'manual', 'private',
       'visibility', 'flagged', 'gear_id', 'start_latlng', 'end_latlng',
       'average_speed', 'max_speed', 'has_heartrate', 'heartrate_opt_out',
       'display_hide_heartrate_option', 'elev_high', 'elev_low', 'upload_id',
       'upload_id_str', 'external_id', 'from_accepted_tag', 'pr_count',
       'total_photo_count', 'has_kudoed', 'suffer_score', 'average_watts',
       'kilojoules', 'device_watts'],
      dtype='object')

In [167]:
run_data.head()

,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,...,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,average_watts,kilojoules,device_watts
0,2,"{'id': 8586088, 'resource_state': 1}",RRG : Rush Trail to Gray’s Arch Loop,6312.2,2374,2448,173.8,Run,Run,0.0,...,10150965476,3A315073-851C-4478-8D23-F4D440BAA704-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN
1,2,"{'id': 8586088, 'resource_state': 1}",PMRP : Lode Loop and Drive By Loop,5532.4,1884,1967,124.0,Run,Run,0.0,...,10143932040,7D43355C-FECB-473B-8263-E7C4BDB870DE-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN
2,2,"{'id': 8586088, 'resource_state': 1}",PMRP : Lode Loop and Sore Heel loop,7940.5,2861,2886,153.7,Run,Run,0.0,...,10131336663,4256B6D8-14F3-4F18-8DF3-440342D551CA-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN
3,2,"{'id': 8586088, 'resource_state': 1}",PMRP : Sore Heel Loop,7090.5,2341,2365,210.0,Run,Run,0.0,...,10124522361,0B1C68DC-4E71-4EBB-8217-F688A9582DCD-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN
4,2,"{'id': 8586088, 'resource_state': 1}",Ten Sleep : French Cattle Ranch Approach,1594.6,1483,1483,186.4,Hike,Hike,NaN,...,10023599286,70EA4DF3-F218-4DB7-B818-E242D7EC3F45-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN


In [168]:
columns_to_drop = ['athlete',
                   'resource_state', 
                   'sport_type', 
                   'workout_type',
                   'location_city',
                   'location_state',
                   'location_country', 
                   'kudos_count', 
                   'comment_count', 
                   'athlete_count', 
                   'photo_count', 
                   'trainer', 
                   'commute', 
                   'manual', 
                   'private',
                   'visibility', 
                   'flagged', 
                   'gear_id', 
                   'has_heartrate', 
                   'heartrate_opt_out', 
                   'display_hide_heartrate_option', 
                   'from_accepted_tag', 
                   'total_photo_count', 
                   'has_kudoed', 
                   'average_watts', 
                   'kilojoules',
                   'achievement_count',
                   'device_watts',
                   'upload_id_str',
                   'upload_id',
                   'external_id', 
                   'suffer_score']

run_data.drop(columns=columns_to_drop, inplace=True)

In [169]:
run_data.columns

Index(['name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'id', 'start_date', 'start_date_local',
       'timezone', 'utc_offset', 'map', 'start_latlng', 'end_latlng',
       'average_speed', 'max_speed', 'elev_high', 'elev_low', 'pr_count'],
      dtype='object')

# We only want the activity of running

In [170]:
run_data = run_data.loc[run_data['type'] == 'Run']

In [171]:
run_data.columns

Index(['name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'id', 'start_date', 'start_date_local',
       'timezone', 'utc_offset', 'map', 'start_latlng', 'end_latlng',
       'average_speed', 'max_speed', 'elev_high', 'elev_low', 'pr_count'],
      dtype='object')

In [172]:
map_data = pd.DataFrame(run_data['map'].tolist())
#clean the id character of the a so it can align with original dataframe
map_data['id'] = map_data['id'].str.slice(start=1)
map_data

,id,summary_polyline,resource_state
0,9464901645,qjweFflr}NEAOOI_@Xi@VUJg@@_@zAmDQWOG?IKMG]MQIY...,2
1,9458318004,aowdFvk}}N@KAEB?L_@Gg@CEOGEDI@ABE?MPAB@FCGFG@M...,2
2,9446383475,onwdFzl}}NI{@Dw@GSm@FMTBW\YH]Cm@PY?ULY?OLK@_@K...,2
3,9439898989,inwdFfm}}NBFONWHKEAEQ?QHWb@UBMd@qA`BQZK\GZKPER...,2
4,9294421659,sn`lG`~~mSBBIUG_@K[A?OYE?Is@k@qB_@{@Om@Ys@MQOo...,2
5,9294421645,_w_lGlyanSCMIu@S{@EGB[Ik@AYIIIU@MEEIc@CE?GOQGe...,2
6,9176575201,ejweFrlr}NIQWOQAAEIAIKE?@FFHOOe@BGEKHSGQ?k@SOO...,2
7,9159069458,{mwdFhl}}NEXUXo@?[^YPKCQt@eBvBeA|CY^Y|@GCQfAMD...,2
8,9059541871,unwdFhj}}NACDG?GOa@M??HE@@EC?EHKHEAAFQHO?AACDW...,2
9,9017048707,kowdFvm}}Nu@VuClDm@bBWTMv@MA[p@QhAOPD\m@r@[tAK...,2


In [173]:
#Create a new DataFrame to store decoded polyline
decoded_df = pd.DataFrame(columns=['id', 'decoded_polyline'])

#loop through each row in the DataFrame
for index, row in map_data.iterrows():
    polyline_str = row['summary_polyline']
    decoded_polyline = polyline.decode(polyline_str)

    #appened the decoded polyline to the new DataFrame
    decoded_df = decoded_df.append({'id' : row['id'], 'decoded_polyline' : decoded_polyline}, ignore_index=True)

decoded_df

/var/folders/02/lzvd5scj4zb274qzdzm7r7p80000gn/T/ipykernel_5879/1457619382.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  decoded_df = decoded_df.append({'id' : row['id'], 'decoded_polyline' : decoded_polyline}, ignore_index=True)
/var/folders/02/lzvd5scj4zb274qzdzm7r7p80000gn/T/ipykernel_5879/1457619382.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  decoded_df = decoded_df.append({'id' : row['id'], 'decoded_polyline' : decoded_polyline}, ignore_index=True)
/var/folders/02/lzvd5scj4zb274qzdzm7r7p80000gn/T/ipykernel_5879/1457619382.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  decoded_df = decoded_df.append({'id' : row['id'], 'decoded_polyline' : decoded_polyline}, ignore_index=True)
/var/folders/02/lzvd5s

,id,decoded_polyline
0,9464901645,"[(37.80793, -83.6578), (37.80796, -83.65779), ..."
1,9458318004,"[(37.64481, -83.71404), (37.6448, -83.71398), ..."
2,9446383475,"[(37.64472, -83.71422), (37.64477, -83.71392),..."
3,9439898989,"[(37.64469, -83.71428), (37.64467, -83.71432),..."
4,9294421659,"[(44.08058, -107.31505), (44.08056, -107.31507..."
5,9294421645,"[(44.0768, -107.32967), (44.07682, -107.3296),..."
6,9176575201,"[(37.80787, -83.65786), (37.80792, -83.65777),..."
7,9159069458,"[(37.64462, -83.71413), (37.64465, -83.71426),..."
8,9059541871,"[(37.64475, -83.71381), (37.64476, -83.71379),..."
9,9017048707,"[(37.64486, -83.71436), (37.64513, -83.71448),..."


In [174]:
run_data.columns

Index(['name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'id', 'start_date', 'start_date_local',
       'timezone', 'utc_offset', 'map', 'start_latlng', 'end_latlng',
       'average_speed', 'max_speed', 'elev_high', 'elev_low', 'pr_count'],
      dtype='object')

## Conversions for Metrics

In [175]:
run_data['distance_miles'] = round(run_data['distance'] * 0.00062137119, 2)
run_data['moving_time_minutes'] = round(run_data['moving_time'] / 60, 2)
run_data['moving_time_hours'] = round(run_data['moving_time'] / 3600, 2)

In [176]:
average_distance_miles = round(run_data['distance_miles'].mean(), 2)
print("Average Distance (miles):", average_distance_miles)

average_time_minutes = round(run_data['moving_time_minutes'].mean(), 2)
print("Average Time Ran (minutes):", average_time_minutes)

max_distance_ran = round(run_data['distance_miles'].max(), 2)
print("Longest Run:", max_distance_ran, "miles")

max_duration_mintues = round(run_data['moving_time_minutes'].max(), 2)
max_duration_hours = round(run_data['moving_time_hours'].max(), 2)
print("Longest Duration:", max_duration_mintues,"minutes. Converted to hours:", max_duration_hours)

total_distance_miles = round(run_data['distance'].sum() * 0.00062137119, 2)
print("Total Distance Covered to the date (miles):", total_distance_miles)

Average Distance (miles): 3.98
Average Time Ran (minutes): 40.56
Longest Run: 9.62 miles
Longest Duration: 117.57 minutes. Converted to hours: 1.96
Total Distance Covered to the date (miles): 147.18


In [177]:
run_data['name']

0              RRG : Rush Trail to Gray’s Arch Loop
1                PMRP : Lode Loop and Drive By Loop
2               PMRP : Lode Loop and Sore Heel loop
3                             PMRP : Sore Heel Loop
6                           Ten Sleep : Morning Run
7                           Ten Sleep : Morning Run
8               RRG : Grays Arch Loop w/ Rush Trail
9           PMRP : Sore Heel Loop and Drive By Loop
10                   PMRP : Lode Loop and Lode Hill
11            PMRP : Lode Loop and Flat Holler Loop
13               PMRP : Drive By Loop and Lode Loop
15                            PMRP : Sore Heel Loop
16                            RRG : Courthouse Loop
17                       LOUISVILLE : Cherokee Park
20                            RRG : Courthouse Loop
21                            PMRP : Sore Heel Loop
23                            PMRP : Sore Heel Loop
24                            PMRP : Sore Heel Loop
25                            PMRP : Sore Heel Loop
26          

# Going further into the data to only collect runs from RRGCC land

In [178]:
pmrp_run_data = run_data[run_data['name'].str.contains('PMRP', case=False, na=False)]
rrg_run_data = run_data[run_data['name'].str.contains('RRG', case=False, na=False)]
sore_heel_data = run_data[run_data['name'].str.contains('Sore Heel', case=False, na=False)]
lode_loop_data = run_data[run_data['name'].str.contains('Lode Loop', case=False, na=False)]
drive_by_loop_data = run_data[run_data['name'].str.contains('Drive By Loop', case=False,na=False)]

In [179]:
all_csv_path = 'csv/run/all_run_data.csv'
pmrp_csv_path = 'csv/run/pmrp_run_data.csv'
rrg_csv_path = 'csv/run/rrg_run_data.csv'
sore_heel_csv_path = 'csv/run/sore_heel_data.csv'
lode_loop_csv_path = 'csv/run/lode_loop_data.csv'
drive_by_csv_path = 'csv/run/drive_by_data.csv'
polyline_csv_path = 'csv/run/polyline.csv'

run_data.to_csv(all_csv_path, index=False)
pmrp_run_data.to_csv(pmrp_csv_path, index=False)
rrg_run_data.to_csv(rrg_csv_path, index=False)
sore_heel_data.to_csv(sore_heel_csv_path, index=False)
lode_loop_data.to_csv(lode_loop_csv_path, index=False)
drive_by_loop_data.to_csv(drive_by_csv_path, index=False)
decoded_df.to_csv(polyline_csv_path, index=False)